# TOKENIZER

This class takes in the File Name of a movie review and converts it into a Bag of Words using the imdb.vocab file.

RETURNS a string with 0 at the front and a count of each word present afterwards, delimited by whitespace. No newline character at the end.

In [1]:
import string
import re

def Tokenizer(file_name):
    vocab = open("aclImdb/imdb.vocab").read().splitlines()
    
    review = open(file_name).read()
    allow = string.ascii_letters + string.whitespace + "'-"
    review = re.sub('[^%s]' % allow, '', review).lower().split()
    
    word_count = {}
    
    for word in review:
        count = review.count(word)
        word_count[word] = count
        
    converted_word_count = {}
    
    for word, count in word_count.items():
        try:
            index = vocab.index(word)
            converted_word_count[index] = count
        except ValueError:    
            continue
            
    indices = list(converted_word_count.keys())
    indices.sort()
    
    sorted_bow = {}
    
    for index in indices:
        sorted_bow[index] = converted_word_count[index]
        
    return_string = "0 "
    
    for index, count in sorted_bow.items():
        return_string += str(index)
        return_string += ":"
        return_string += str(count)
        return_string += " "
    
    return return_string

In [2]:
Tokenizer("0_9.txt")

'0 0:9 1:1 2:4 3:4 4:6 5:4 6:2 7:2 8:4 10:4 12:2 26:1 28:1 29:2 32:1 41:1 45:1 47:1 50:1 54:2 57:1 59:1 63:2 64:1 66:1 68:2 70:1 72:1 78:1 100:1 106:1 116:1 122:1 125:1 136:1 140:1 142:1 150:1 167:1 183:1 201:1 207:1 208:1 213:1 217:1 230:1 255:1 321:4 343:1 357:1 370:1 390:2 468:1 514:1 571:1 619:1 671:1 766:1 877:1 1057:1 1179:1 1192:1 1402:2 1416:1 1477:2 1940:1 1941:1 2096:1 2243:1 2285:1 2379:1 2934:1 2938:1 3520:1 3647:1 4938:1 5138:3 5715:1 5726:1 5731:1 5812:1 8319:1 8567:1 10480:1 14239:1 20604:1 22409:4 24551:1 47304:1 '